In [7]:
# Importing Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import KBinsDiscretizer

# Table 1 Corona Index 
table_corona = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')
# Table 2 Freedom Index 
table_freedom = pd.read_csv('modified_data/table_freedom_press.csv',decimal=',')
# Table 3 GDP index
table_gdp = pd.read_csv('modified_data/table_gdp.csv')
# Table 4 Population Index 
table_population = pd.read_csv('modified_data/table_population.csv')
# Table 5 Human Freedom
table_hfi = pd.read_csv('modified_data/table_hfi.csv', na_values='-')
# Table 6 Health Expenditure Index created in nha2.csv
table_health_expenditure = pd.read_csv('modified_data/table_health_expenditure.csv')

# NOTE: only Table 1 will replace with the script because we get table 1 from the internet. 
# Other tables come from the local file. We can edit their values in the file manually. 
# However, table 1 comes from online they have to be modified. Otherwise, we have to adjust country values every time manually when data is changed by online.
table_corona = table_corona.replace(['Brunei'], 'Brunei Darussalam')
table_corona = table_corona.replace(['Czechia'], 'Czech Rep.')
table_corona = table_corona.replace(['Democratic Republic of Congo'], 'DR Congo')
table_corona = table_corona.replace(['Cote d\'Ivoire'], 'Ivory Coast')
table_corona = table_corona.replace(['South Korea'],'Korea, South')
table_corona = table_corona.replace(['Kyrgyzstan'], 'Kyrgyz Republic')
table_corona = table_corona.replace(['Slovakia'],'Slovak Rep.')
table_corona = table_corona.replace(['Timor'],'Timor-Leste')

# Delete unncesary features
del table_corona['iso_code']
del table_corona['date']
del table_corona['positive_rate']

# Delete some features due to not enough information may be in future it would be added if enough data is added from internet
del table_corona['icu_patients']
del table_corona['icu_patients_per_million']
del table_corona['hosp_patients']
del table_corona['hosp_patients_per_million']
del table_corona['total_vaccinations']
del table_corona['total_vaccinations_per_hundred']
del table_corona['handwashing_facilities']

# Groupby data 'location' and aggregate important columns 

table_corona_grouped = table_corona.groupby(['location']).agg({'total_cases':'last',\
                                             'total_deaths': 'last',\
                                             'total_cases_per_million': 'last',\
                                             'total_deaths_per_million': 'last', \
                                             'reproduction_rate': 'mean', \
                                             'stringency_index': 'mean',\
                                             'median_age': 'mean', \
                                             'aged_65_older': 'last', \
                                             'aged_70_older': 'last', \
                                             'extreme_poverty': 'last',\
                                             'cardiovasc_death_rate': 'last', \
                                             'diabetes_prevalence': 'last', \
                                             'female_smokers': 'mean',\
                                             'male_smokers': 'mean', \
                                             'hospital_beds_per_thousand':'mean',\
                                             'life_expectancy': 'last', \
                                             'human_development_index': 'mean',\
                                             })

# Somehow when I save the table location attribute appears however when it is grouped location attribute disappears
table_corona_grouped.to_csv('modified_data/table_corona_grouped.csv')
table_corona_grouped = pd.read_csv('modified_data/table_corona_grouped.csv')


# Feature Edit Table 2
del table_freedom['ISO']
del table_freedom['FR_Country']
del table_freedom['ES_country']
del table_freedom['Zone']
del table_freedom['AR_country']
del table_freedom['FA_country']

# Feature Edit Table 5
# Get only 2017 values for uptodate
table_hfi = table_hfi[(table_hfi['year'] == 2017)]

# Remove values has all NaN values 
del table_hfi['pf_association_political_establish']  # cols don't have any entrires (156 Nan)
del table_hfi['pf_association_prof_establish']       #
del table_hfi['pf_association_prof_operate']         #
del table_hfi['pf_religion_estop_operate']           #
del table_hfi['pf_religion_estop_establish']         #
del table_hfi['pf_ss_women_inheritance_widows']      #
del table_hfi['pf_ss_women_inheritance_daughters']   #
del table_hfi['pf_association_sport_establish']      #
del table_hfi['pf_association_political_operate']    #
del table_hfi['pf_association_sport_operate']        #

# remove unnecessary features
del table_hfi['year']
del table_hfi['ISO_code']
del table_hfi['hf_rank']
del table_hfi['ef_rank']
del table_hfi['pf_rank']

# Somehow ef_regulation_labor_dismissal coloum is object convert float
table_hfi['ef_regulation_labor_dismissal'] = pd.to_numeric(table_hfi['ef_regulation_labor_dismissal'])

# Get mean values for Nan values by getting each region's means by column to eliminate all Null values.
row_region_means = pd.Series([], dtype=pd.StringDtype())

for index_num, row in table_hfi.iterrows():
    for col, value in row.items():
        if pd.isna(value):
            table_hfi.loc[index_num, (col)] = table_hfi[(table_hfi['region'] == row['region'])].mean()[col]

# Feature Edit Table 6
del table_health_expenditure['Unnamed: 0']
  

# Create Filter. If we filter by another table, there may be some data that does not include in other tables.
list_filter = list(set(table_hfi['countries']))

# Filter all tables 

# Table 1 filtering
table_filtered_grouped_corona = table_corona_grouped[table_corona_grouped['location'].isin(list_filter)]
table_filtered_grouped_corona = table_filtered_grouped_corona.reset_index(drop=True)

# Table 2 filtering
table_filtered_freedom = table_freedom[table_freedom['EN_country'].isin(list_filter)]
table_filtered_freedom = table_filtered_freedom.reset_index(drop=True)

# Table 3 filtering
table_filtered_gdp = table_gdp[table_gdp['country'].isin(list_filter)]
table_filtered_gdp = table_filtered_gdp.reset_index(drop=True)

# Table 4 filternig
table_filtered_population = table_population[table_population['name'].isin(list_filter)]
table_filtered_population = table_filtered_population.reset_index(drop=True)

# Table 5 filtering
table_filtered_hfi = table_hfi[table_hfi['countries'].isin(list_filter)]
table_filtered_hfi = table_filtered_hfi.reset_index(drop=True)

# Table 6 filtering
table_filtered_health_expenditure = table_health_expenditure[table_health_expenditure['Countries'].isin(list_filter)]
table_filtered_health_expenditure = table_filtered_health_expenditure.reset_index(drop=True)

# Sort and reset indexes for concatination
table_filtered_grouped_corona = table_filtered_grouped_corona.sort_values('location').reset_index(drop=True)
table_filtered_freedom = table_filtered_freedom.sort_values('EN_country').reset_index(drop=True)
table_filtered_gdp = table_filtered_gdp.sort_values('country').reset_index(drop=True)
table_filtered_population = table_filtered_population.sort_values('name').reset_index(drop=True)
table_filtered_health_expenditure = table_filtered_health_expenditure.sort_values('Countries').reset_index(drop=True)
table_filtered_hfi = table_filtered_hfi.sort_values('countries').reset_index(drop=True)

# Check if row numbers are equal when a full table is sorted and filtered. 
# If all filtered rows are not equal, then print an error tables row number are not equal. 
# Rows have to be equal when append tables; otherwise, the data could be wrong or corrupted
if (len(table_filtered_grouped_corona.index) == len(table_filtered_freedom.index) == len(table_filtered_gdp.index) ==\
   len(table_filtered_population.index) == len(table_filtered_health_expenditure.index) == len(table_filtered_hfi.index)):
    
    # Feature Selection
    feature_corana = table_filtered_grouped_corona.columns.values.tolist()
    feature_freedom = ['Score A', 'Sco Exa', 'Score 2020', 'Progression RANK']
    feature_gdp = ['imfGDP', 'unGDP', 'gdpPerCapita']
    feature_population = ['pop2020', 'area', 'Density', 'GrowthRate']
    feature_health_expenditure = ['current_health_expenditure_average']
    feature_hfi = table_filtered_hfi.columns.values.tolist()

    table_filtered_grouped_selected_corona = table_filtered_grouped_corona[feature_corana]
    table_filtered_selected_freedom = table_filtered_freedom[feature_freedom]
    table_filtered_selected_gdp = table_filtered_gdp[feature_gdp]
    table_filtered_selected_population = table_filtered_population[feature_population]
    table_filtered_selected_health_expenditure = table_filtered_health_expenditure[feature_health_expenditure]
    table_filtered_selected_hfi = table_filtered_hfi[feature_hfi]

    table_full = pd.concat([table_filtered_grouped_selected_corona, table_filtered_selected_freedom, table_filtered_selected_gdp, \
                            table_filtered_selected_population, table_filtered_selected_health_expenditure, table_filtered_selected_hfi], \
                           axis=1, verify_integrity=True)
    
    # Fix table_full some NaN values by replacing zeros
    # Some countries don't have deaths
    table_full['total_deaths'] = table_full['total_deaths'].fillna(0)
    table_full['total_deaths_per_million'] = table_full['total_deaths_per_million'].fillna(0)
    
    # Şu kısımda bazı NaN değerleri doldurmaya çalışıyoruz eğer yazılacak herhangibir değer yoksa o değeri siliyorum
    # --------------------------------------------------------------------------------------------------------------------
    # Some countries dont have exterme poverty or their values just old
    # Replace NaN values with old values and Zeros
    bahrain = table_full['countries'].str.startswith('Bahrain')
    table_full.loc[bahrain,'extreme_poverty'] = 0.0
    belarus = table_full['countries'].str.startswith('Belarus')
    table_full.loc[belarus,'extreme_poverty'] = 0.0
    botswana = table_full['countries'].str.startswith('Botswana')
    table_full.loc[botswana,'extreme_poverty'] = 18.2
    capeVerde = table_full['countries'].str.startswith('Cape Verde')
    table_full.loc[capeVerde,'extreme_poverty'] = 8.1
    germany= table_full['countries'].str.startswith('Germany')
    table_full.loc[germany,'extreme_poverty'] = 0.0
    cyprus= table_full['countries'].str.startswith('Cyprus')
    table_full.loc[cyprus,'extreme_poverty'] = 0.0
    czechia= table_full['countries'].str.startswith('Czechia')
    table_full.loc[czechia,'extreme_poverty'] = 0.0
    eswatini= table_full['countries'].str.startswith('Eswatini')
    table_full.loc[eswatini,'extreme_poverty'] = 42.0
    finland= table_full['countries'].str.startswith('Finland')
    table_full.loc[finland,'extreme_poverty'] = 0.0
    france= table_full['countries'].str.startswith('France')
    table_full.loc[france,'extreme_poverty'] = 0.0
    jamaica=table_full['countries'].str.startswith('Jamaica')
    table_full.loc[jamaica,'extreme_poverty'] = 1.7
    japan=table_full['countries'].str.startswith('Japan')
    table_full.loc[japan,'extreme_poverty'] = 0.2
    lebanon= table_full['countries'].str.startswith('Lebanon')
    table_full.loc[lebanon,'extreme_poverty'] = 0.0
    mali=table_full['countries'].str.startswith('Mali')
    table_full.loc[mali,'extreme_poverty'] = 49.7
    netherlands=table_full['countries'].str.startswith('Netherlands')
    table_full.loc[netherlands,'extreme_poverty'] = 0.0
    slovenia= table_full['countries'].str.startswith('Slovenia')
    table_full.loc[slovenia,'extreme_poverty'] = 0.0
    suriname=table_full['countries'].str.startswith('Suriname')
    table_full.loc[suriname,'extreme_poverty'] = 23.4
    switzerland=table_full['countries'].str.startswith('Switzerland')
    table_full.loc[switzerland,'extreme_poverty'] = 0.0
    newZeland=table_full['countries'].str.startswith('New Zeland')
    table_full.loc[newZeland,'extreme_poverty'] = 0.0
    qatar=table_full['countries'].str.startswith('Qatar')
    table_full.loc[qatar,'extreme_poverty'] = 0.0
    singapore=table_full['countries'].str.startswith('Singapore')
    table_full.loc[singapore,'extreme_poverty'] = 0.0
    poland=table_full['countries'].str.startswith('Poland')
    table_full.loc[poland,'extreme_poverty'] = 0.0
   
     # Drop some countries by NaN values
    table_full = table_full.dropna(subset=['male_smokers'])
    table_full = table_full.dropna(subset=['hospital_beds_per_thousand'])
    table_full = table_full.dropna(subset=['stringency_index'])
    table_full = table_full.dropna(subset=['aged_70_older'])
    table_full = table_full.dropna(subset=['extreme_poverty'])
    table_full = table_full.dropna(subset=['reproduction_rate'])

    del table_full['location']
    # --------------------------------------------------------------------------------------------------------------------               
   
    
    # Save the table
    table_full = table_full.reset_index(drop=True)
    table_full.to_csv('table_full.csv')

else:
    print('Rows are not equal table is not created !!!')

In [2]:
# Bütün sütunlarta bak ve NaN olan ülkeleri değerleri ile yazdır

def table_full_findNull():
    for col in list(table_full.columns.values):
        if table_full[col].isna().any() == True: 
            print('col name:', col)
            print(table_full[pd.isnull(table_full[col])][['countries', col]])
    
table_full_findNull()

In [6]:
pd.options.display.max_rows = 200
table_full.dtypes

total_cases                           float64
total_deaths                          float64
total_cases_per_million               float64
total_deaths_per_million              float64
reproduction_rate                     float64
stringency_index                      float64
median_age                            float64
aged_65_older                         float64
aged_70_older                         float64
extreme_poverty                       float64
cardiovasc_death_rate                 float64
diabetes_prevalence                   float64
female_smokers                        float64
male_smokers                          float64
hospital_beds_per_thousand            float64
life_expectancy                       float64
human_development_index               float64
Score A                               float64
Sco Exa                               float64
Score 2020                            float64
Progression RANK                        int64
imfGDP                            